In [ ]:
!pip install pyLDAvis
import nltk
nltk.download('punkt')
from nltk import sent_tokenize,word_tokenize
from nltk import regexp_tokenize
from nltk.corpus import stopwords
from nltk.corpus import wordnet


import matplotlib.pyplot as plt
import gensim
import numpy as np
import spacy

from gensim.models import CoherenceModel, LdaModel, LsiModel, HdpModel
from gensim.models.wrappers import LdaMallet
from gensim.corpora import Dictionary
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()


import os, re, operator, warnings
warnings.filterwarnings('ignore')
%matplotlib inline


nltk.download('stopwords')
nltk.download('wordnet')
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Iterable


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
df = pd.read_csv("/content/drive/MyDrive/us_president_speeches/presidential_speeches.csv")

In [ ]:
df.isnull().sum()
df['Transcript'] = df['Transcript'].fillna('Not Available')

In [ ]:
df['President'].value_counts(sort =True,ascending =False)

Lyndon B. Johnson        71
Ronald Reagan            59
Barack Obama             50
Franklin D. Roosevelt    49
John F. Kennedy          44
George W. Bush           39
Bill Clinton             39
Woodrow Wilson           33
Ulysses S. Grant         32
Andrew Johnson           31
Herbert Hoover           30
Grover Cleveland         30
Andrew Jackson           26
James K. Polk            25
Thomas Jefferson         24
Benjamin Harrison        23
Richard M. Nixon         23
George H. W. Bush        23
Jimmy Carter             22
James Madison            22
Theodore Roosevelt       22
George Washington        21
Donald Trump             19
Harry S. Truman          19
John Tyler               18
Warren G. Harding        18
Rutherford B. Hayes      16
Franklin Pierce          15
Abraham Lincoln          15
James Buchanan           14
William McKinley         14
Gerald Ford              14
William Taft             12
Calvin Coolidge          12
Chester A. Arthur        11
James Monroe        

In [ ]:
num_speeches = df['President'].value_counts(sort =True,ascending =False).values
print(len(num_speeches))
print(sum(num_speeches))
print("Average number of speeches by presidents:",sum(num_speeches)/len(num_speeches))

44
992
Average number of speeches by presidents: 22.545454545454547


In [ ]:
df.head()

,Date,President,Party,Speech Title,Summary,Transcript,URL
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...


In [ ]:
import re
import string

def dataframe_clean(text):
   
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

abc = lambda x: dataframe_clean(x)


In [ ]:
df['transcript_clean'] = df['Transcript'].apply(abc)
df.head()

,Date,President,Party,Speech Title,Summary,Transcript,URL,transcript_clean
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and the house of...
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...,whereas it is the duty of all nations to ackno...
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and house of rep...
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and house of rep...
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...,i the president of the united states by my own...


In [ ]:
from tqdm.notebook import tqdm
from tqdm import tqdm
df_clean = df
import spacy
nlp = spacy.load('en')


def generate_tokens(df,column):
  skl_text = []
  line = []
  for i in tqdm(df[column]):
    doc = nlp(i.lower())
    for word in doc:
        if (not word.is_stop and len(word)>2 ):
            line.append(word.lemma_)
    skl_text.append(line)
    line = []
  return skl_text


In [ ]:
df_clean['transcript_tokens'] = generate_tokens(df_clean,'transcript_clean')
df_clean.head()

100%|██████████| 992/992 [09:20<00:00,  1.77it/s]


,Date,President,Party,Speech Title,Summary,Transcript,URL,transcript_clean,transcript_tokens
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and the house of...,"[fellow, citizen, senate, house, representativ..."
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...,whereas it is the duty of all nations to ackno...,"[duty, nation, acknowledge, providence, almigh..."
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and house of rep...,"[fellow, citizen, senate, house, representativ..."
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and house of rep...,"[fellow, citizen, senate, house, representativ..."
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...,i the president of the united states by my own...,"[president, united, states, mouth, write, spee..."


In [ ]:
stopp =stopwords.words('english')
# mostf = ['unknown','day','good','trump','new','question','season','president','say']
# for i in mostf:
#   stopp.append(i)
df_clean['transcript_tokens'] = df_clean['transcript_tokens'].apply(lambda x: [item for item in (x) if item not in stopp and len(item) >2])
df_clean.head()

,Date,President,Party,Speech Title,Summary,Transcript,URL,transcript_clean,transcript_tokens
0,1789-04-30,George Washington,Unaffiliated,First Inaugural Address,Washington calls on Congress to avoid local an...,Fellow Citizens of the Senate and the House of...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and the house of...,"[fellow, citizen, senate, house, representativ..."
1,1789-10-03,George Washington,Unaffiliated,Thanksgiving Proclamation,"At the request of Congress, Washington establi...",Whereas it is the duty of all Nations to ackno...,https://millercenter.org/the-presidency/presid...,whereas it is the duty of all nations to ackno...,"[duty, nation, acknowledge, providence, almigh..."
2,1790-01-08,George Washington,Unaffiliated,First Annual Message to Congress,"In a wide ranging speech, President Washington...",Fellow Citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and house of rep...,"[fellow, citizen, senate, house, representativ..."
3,1790-12-08,George Washington,Unaffiliated,Second Annual Message to Congress,Washington focuses on commerce in his second a...,Fellow citizens of the Senate and House of Rep...,https://millercenter.org/the-presidency/presid...,fellow citizens of the senate and house of rep...,"[fellow, citizen, senate, house, representativ..."
4,1790-12-29,George Washington,Unaffiliated,Talk to the Chiefs and Counselors of the Senec...,The President reassures the Seneca Nation that...,"I the President of the United States, by my ow...",https://millercenter.org/the-presidency/presid...,i the president of the united states by my own...,"[president, united, states, mouth, write, spee..."


In [ ]:
speech_len = []
for i in df_clean['transcript_tokens']:
  speech_len.append(len(i))
print("Average number of words in each speech:",sum(speech_len)/len(speech_len))

Average number of words in each speech: 1621.1683467741937


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.decomposition import TruncatedSVD
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from sklearn.decomposition import LatentDirichletAllocation
from gensim import matutils, models
import scipy.sparse
from gensim import models,corpora

In [ ]:

def data_prepare_topic(df,column):
  bigram = gensim.models.Phrases(df[column])
  texts = [bigram[line] for line in df[column]]



  dictionary = Dictionary(texts)
  print(dictionary.items())
  corpus = [dictionary.doc2bow(text) for text in texts]
  return corpus, dictionary




In [ ]:
def show_topics(df,column,corpus,dictionary):

  tfidf = models.TfidfModel(corpus)
  transformed_tfidf = tfidf[corpus]

  lda1 = models.LdaMulticore(corpus = transformed_tfidf,id2word= dictionary,num_topics=4,workers = 4,passes = 4,iterations = 5)




  # tfidf_vectorizer = TfidfVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
  # tfidf = tfidf_vectorizer.fit_transform(df[column])
  # tfidf_feature_names = tfidf_vectorizer.get_feature_names()

  # tf_vectorizer = CountVectorizer(max_df=0.95, min_df=2, max_features=no_features, stop_words='english')
  # tf = tf_vectorizer.fit_transform(df[column])

  # tf_feature_names = tf_vectorizer.get_feature_names()

  # no_topics = 10

  # # Run NMF
  # #nmf = NMF(n_components=no_topics, random_state=1, alpha=.1, l1_ratio=.5, init='nndsvd').fit(tfidf)

  # # Run LDA
  # lda1 = models.LdaModel(corpus = tfidf_feature_names,id2word= dictionary,num_topics=5, passes=200)
  # # lda1.show_topics()

  # no_top_words = 10
  return lda1,corpus,dictionary

In [ ]:
president_options = df_clean["President"].value_counts().keys()


perplexity= []
coherence = []

from tqdm import tqdm
for i in tqdm(president_options):
  df_president = df_clean.loc[df_clean['President'] == i]
  corpus_bow, dictionary_bow = data_prepare_topic(df_president,'transcript_tokens')
  lda1,corpus,dictionary = show_topics(df_president,'transcript_clean',corpus_bow,dictionary_bow)
  perplexity.append(lda1.log_perplexity(corpus))
  coherence_model = CoherenceModel(model = lda1,texts = df_president['transcript_tokens'],
                                   dictionary = dictionary_bow,coherence = 'c_v')
  coherence_lda = coherence_model.get_coherence()
  coherence.append(coherence_lda)



  0%|          | 0/44 [00:00<?, ?it/s]

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f83facab7d0>)


  2%|▏         | 1/44 [00:05<04:08,  5.77s/it]

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f83faabb390>)


  5%|▍         | 2/44 [00:10<03:45,  5.36s/it]

...

 95%|█████████▌| 42/44 [01:47<00:02,  1.14s/it]

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f83f89e0bd0>)


 98%|█████████▊| 43/44 [01:47<00:00,  1.11it/s]

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f83faae2f10>)


100%|██████████| 44/44 [01:48<00:00,  2.46s/it]


In [ ]:
df_topic_scores = pd.DataFrame()
df_topic_scores['President'] = president_options
df_topic_scores['Perplexity'] = perplexity
df_topic_scores['Coherence'] = coherence
df_topic_scores

,President,Perplexity,Coherence
0,Lyndon B. Johnson,-10.239212,NaN
1,Ronald Reagan,-10.718421,NaN
2,Barack Obama,-10.809462,NaN
3,Franklin D. Roosevelt,-10.196152,NaN
4,John F. Kennedy,-10.451707,NaN
5,George W. Bush,-10.358317,NaN
6,Bill Clinton,-10.528386,NaN
7,Woodrow Wilson,-10.204755,NaN
8,Ulysses S. Grant,-10.314885,NaN
9,Andrew Johnson,-9.994403,NaN


In [ ]:
df_president = df_clean.loc[df_clean['President'] == 'Richard M. Nixon']
corpus_bow, dictionary_bow = data_prepare_topic(df_president,'transcript_tokens')
lda1,corpus,dictionary = show_topics(df_president,'transcript_clean',corpus_bow,dictionary_bow)
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(lda1, corpus, dictionary)

ItemsView(<gensim.corpora.dictionary.Dictionary object at 0x7f7250804090>)


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.001304 -0.000104       1        1  64.139881
1     -0.000088  0.000433       2        1  27.956121
2     -0.000432 -0.000236       3        1   4.990630
3     -0.000784 -0.000092       4        1   2.913367, topic_info=                  Term       Freq      Total Category  logprob  loglift
1881          cambodia  10.000000  10.000000  Default  30.0000  30.0000
3073         watergate  13.000000  13.000000  Default  29.0000  29.0000
1465        withdrawal  14.000000  14.000000  Default  28.0000  28.0000
1399  south_vietnamese  12.000000  12.000000  Default  27.0000  27.0000
119           congress  12.000000  12.000000  Default  26.0000  26.0000
...                ...        ...        ...      ...      ...      ...
1448           vietnam   0.236800  16.956868   Topic4  -8.0748  -0.7354
206               fact   0.230820  10.606306   Topic4  -8.1004  -0.2917
741              enemy   0.231331  11.579846   Topic4  -8.0982  -0.3773
418            politic   0.228490   9.598659   Topic4  -8.1106  -0.2020
1398     south_vietnam   0.229083  17.228703   Topic4  -8.1080  -0.7844

[349 rows x 6 columns], token_table=      Topic      Freq       Term
term                            
3660      1  0.610631  abhorrent
3660      2  0.305315  abhorrent
2532      1  0.814257       abms
2532      2  0.180946       abms
626       1  0.591034   abundant
...     ...       ...        ...
3658      2  0.323320     wrench
622       1  0.664589       year
622       2  0.332295       year
1467      1  0.627666   year_ago
1467      2  0.313833   year_ago

[557 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4])

In [ ]:
!pip install streamlit

     |████████████████████████████████| 9.1 MB 5.0 MB/s 
     |████████████████████████████████| 111 kB 46.3 MB/s 
     |████████████████████████████████| 180 kB 46.5 MB/s 
     |████████████████████████████████| 178 kB 17.4 MB/s 
     |████████████████████████████████| 4.3 MB 30.8 MB/s 
     |████████████████████████████████| 76 kB 4.3 MB/s 
     |████████████████████████████████| 63 kB 1.6 MB/s 
     |████████████████████████████████| 126 kB 64.9 MB/s 
     |████████████████████████████████| 791 kB 58.6 MB/s 
     |████████████████████████████████| 374 kB 69.5 MB/s 
  Created wheel for pympler: filename=Pympler-0.9-py3-none-any.whl size=164824 sha256=5ac772dcdba37cef1f6cb56293c8ef47ce60a05e6370fcd0aa9aa43010855112
  Stored in directory: /root/.cache/pip/wheels/1a/f3/d8/35d5614ea4ddd295ffb9372a5f2f9570d9593d1ea4be33ec6d
  Created wheel for blinker: filename=blinker-1.4-py3-none-any.whl size=13478 sha256=603b48b38362564b715692491ec50c1b00af1de85c3b4e6c1fc56e627b1325ee
  Stored in direc

In [ ]:
import streamlit as st

ContextualVersionConflict: ignored